In [0]:
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive


In [0]:
drive.mount("/content/drive", force_remount=True)
fma_dir = '/content/drive/My Drive/fma_small'

Mounted at /content/drive


In [0]:
def get_mel_spectrogram(f):
  y, sr = librosa.load(f)
  ms = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=1024)
  ms = librosa.power_to_db(ms, ref=np.max)
  return ms.T

In [0]:
trackdata = pd.read_csv('/content/drive/My Drive/tracks.csv')
headers = trackdata.iloc[0]
headers[0] = 'track_id'
trackdata.columns = headers
trackdata = trackdata.iloc[2:]
trackdata = trackdata[trackdata['subset']=='small']
instrumental_ids = trackdata[trackdata['genre_top']=='Instrumental']['track_id']      #change genre according to requirement. Refer to FMA dataset metadata.
instrumental_filenames = ['{:06d}'.format(int(ids)) for ids in instrumental_ids]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
filenodes = []
for _, direc, files in os.walk(fma_dir):
  if direc == []:
    filenodes.extend(int(filex[:-4]) for filex in files)
    filenamestr = ['{:06d}'.format(filenode) for filenode in filenodes]
    filepaths = [os.path.join(fma_dir, fps[:3], fps + '.mp3') for fps in instrumental_filenames]
   #filepaths = [os.path.join(fma_dir, fps[:3], fps + '.mp3') for fps in filenamestr]
print(len(filepaths))

spectrogram_data = np.empty((0, 640, 128))
complete_spectrogram_data = np.empty((0, 640, 128))
count = 0
for f in filepaths:
  try:
    count += 1
    ms = get_mel_spectrogram(f)
    ms = ms[:640, :] #truncate length to match size(the larger dimension is for time, and we end up trimming all files to same time)
    spectrogram_data = np.append(spectrogram_data, [ms], axis=0)
    complete_spectrogram_data = np.append(complete_spectrogram_data, [ms], axis=0)
    if count%1000==0:
      strname = '/content/drive/My Drive/FilteredDatasets/complete_train_small_fma_at_'+str(count)
      np.savez(strname, spectrogram_data)
      spectrogram_data = np.empty((0, 640, 128))
  except:
    print('Failed: ',count)
    continue

print(len(complete_spectrogram_data))

np.savez('/content/drive/My Drive/FilteredDatasets/complete_train_small_fma_complete', complete_spectrogram_data)


1000
1000
